In [2]:
import pandas as pd
import os
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import geopandas as gpd
import contextily as ctx
import matplotlib as mpl
# from pandarallel import pandarallel

In [ ]:
# # def get_county_report(county_fip):
# county_address_dir = '../data/county_address'
# for file in tqdm(os.listdir(county_address_dir)):
#     test_county= os.path.join(county_address_dir, file)
#     tdf = pd.read_csv(test_county)
#     null_count = tdf[tdf['address'].isnull()].size
#     if null_count > 0:
#         print(file)

# Generate Empty (Go through each county. If there is a file, join on geoid, if there is not, create empty columns)

In [2]:
prefix = 'https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/'
r= requests.get(prefix)
soup = BeautifulSoup(r.content)
valid_zips = {}

for a in soup.find_all('a', href=True):
    if not '.zip' in a['href'] or len(a['href'])!= 28: # filter for the entire state ones
        continue
    valid_zips[a['href'].split('_')[2]] = prefix+a['href']
print(len(valid_zips))

3221


# Plot Jefferson County (01073)

In [ ]:
jc_df = pd.read_csv('../data/county_address/01073.csv.xz', dtype={'geoid20':object})
county_df = gpd.read_file(valid_zips['01073'])

In [ ]:
jc_df = jc_df.rename(columns={'geoid20': 'GEOID20'})
jc_df

In [ ]:
df = pd.merge(jc_df,county_df[['GEOID20']],on='GEOID20', how='right')
df

In [ ]:
df[df['address'].isnull()]

In [ ]:
# county_df.crs= 4326
# ax = county_df.plot(figsize=(10, 10), alpha=0.7, column='address_count', legend=True)
# ctx.add_basemap(ax, zoom=10, crs=county_df.crs.to_string(), source=ctx.providers.Stamen.Toner)

In [24]:
for county in tqdm(valid_zips):
    if os.path.isfile('../data/address/%s.csv.xz' % county):
        # skip if file already exists
        continue
        
    county_shape_df = gpd.read_file(valid_zips[county])    
    if os.path.isfile('../data/county_address/%s.csv.xz' % county):
        cdf = pd.read_csv('../data/county_address/%s.csv.xz' % county, dtype={'geoid20':object})
        cdf = cdf.rename(columns={'geoid20': 'GEOID20'})
        mdf = pd.merge(cdf,county_shape_df[['GEOID20']],on='GEOID20', how='right') # create empty rows based on authority shape fiels
    else:
        mdf = county_shape_df[['GEOID20']]
        mdf = mdf.reindex(columns= ['address', 'GEOID20', 'longitude', 'latitude'])
        assert mdf.size > 16
    mdf.to_csv('../data/address/%s.csv.xz' % county, index =False)

100%|██████████| 3221/3221 [28:24<00:00,  1.89it/s]  


# Generate Report

In [10]:
# # Do some clean up
# d = '../data/address'
# for file in tqdm(os.listdir(d)):
#     df = pd.read_csv(os.path.join(d,file))
#     if not set(df.columns) == set(['address', 'GEOID20', 'longitude', 'latitude']):    
#         df = df.reindex(columns= ['address', 'GEOID20', 'longitude', 'latitude'])
#         df.to_csv(os.path.join(d,file), index =False)

# #     print(df.columns)
# #     null_count =  df[df['address'].isnull()].size
# #     if null_count > 0:
# #         print(file)

100%|██████████| 3221/3221 [32:54<00:00,  1.63it/s]  


In [63]:
# Do some clean up
d = '../data/address'
for file in tqdm(os.listdir(d)):
    df = pd.read_csv(os.path.join(d,file), dtype={'GEOID20': object})
    assert set(df.columns) == set(['address', 'GEOID20', 'longitude', 'latitude'])
    if df.size < 20:
        print('%s: %s' % (file, df.size))
        print(df)
        os.remove(os.path.join(d,file)) 
#     df = df.reindex(columns= ['address', 'GEOID20', 'longitude', 'latitude'])
#     df.to_csv(os.path.join(d,file), index =False)

100%|██████████| 3221/3221 [01:59<00:00, 26.93it/s]


# Generate Report

In [19]:
df['GEOID20'].unique().size

15359

In [27]:
df

,address,GEOID20,longitude,latitude
0,NaN,132839601001038,NaN,NaN
1,NaN,132839601001033,NaN,NaN
2,NaN,132839601002017,NaN,NaN
3,NaN,132839601002007,NaN,NaN
4,NaN,132839601002021,NaN,NaN
...,...,...,...,...
587,NaN,132839601001049,NaN,NaN
588,NaN,132839601001010,NaN,NaN
589,NaN,132839601001011,NaN,NaN
590,"54 spring valley rd, adrian, ga, 31002",132839601001016,-82.560635,32.486963


In [40]:
df['address_count'] = df['address'].apply(lambda x: 0 if pd.isnull(x) else 1)
df

,address,GEOID20,longitude,latitude,address_count
0,NaN,132839601001038,NaN,NaN,0
1,NaN,132839601001033,NaN,NaN,0
2,NaN,132839601002017,NaN,NaN,0
3,NaN,132839601002007,NaN,NaN,0
4,NaN,132839601002021,NaN,NaN,0
...,...,...,...,...,...
587,NaN,132839601001049,NaN,NaN,0
588,NaN,132839601001010,NaN,NaN,0
589,NaN,132839601001011,NaN,NaN,0
590,"54 spring valley rd, adrian, ga, 31002",132839601001016,-82.560635,32.486963,1


In [47]:
tdf = df.groupby('GEOID20')['address_count'].sum().reset_index()
tdf

,GEOID20,address_count
0,132839601001000,0
1,132839601001001,1
2,132839601001002,0
3,132839601001003,1
4,132839601001004,0
...,...,...
543,132839602005104,1
544,132839602005105,0
545,132839602005106,0
546,132839602005107,0


In [48]:
tdf['address_count'].value_counts()

0    463
1     59
2     13
3      9
4      3
5      1
Name: address_count, dtype: int64

In [58]:
df['GEOID20'].unique().size == len(tdf)

True

In [88]:
# Make sure all fips codes are 15 characters long
d = '../data/address'
dfs= []

for file in tqdm(os.listdir(d)):
    df = pd.read_csv(os.path.join(d,file), dtype={'GEOID20': object})
        
    if all(df['GEOID20'].map(len)==15):
        continue
    
    assert all(df['GEOID20'].map(len)==14) # slight fix, fill 0
    df['GEOID20'] = df['GEOID20'].apply(lambda x: x.zfill(15))
    assert all(df['GEOID20'].map(len)==15)
    df.to_csv(os.path.join(d,file))

100%|██████████| 3221/3221 [07:32<00:00,  7.12it/s] 


# Generate Report

In [89]:
d = '../data/address'
dfs= []

for file in tqdm(os.listdir(d)):
    df = pd.read_csv(os.path.join(d,file), dtype={'GEOID20': object})
    
    
    assert all(df['GEOID20'].map(len)==15), print(file)
    
    df['address_count'] = df['address'].apply(lambda x: 0 if pd.isnull(x) else 1)
    report = df.groupby('GEOID20')['address_count'].sum().reset_index()    
    dfs.append(report)

final_df = pd.concat(dfs)
final_df

100%|██████████| 3221/3221 [02:58<00:00, 18.02it/s]


,GEOID20,address_count
0,290854701001000,0
1,290854701001001,0
2,290854701001002,0
3,290854701001003,0
4,290854701001004,0
...,...,...
543,132839602005104,1
544,132839602005105,0
545,132839602005106,0
546,132839602005107,0


In [95]:
final_df = final_df.sort_values(by=['GEOID20', 'address_count'])
final_df

,GEOID20,address_count
0,010010201001000,8
1,010010201001001,10
2,010010201001002,14
3,010010201001003,0
4,010010201001004,0
...,...,...
592,721537506022011,0
593,721537506022012,0
594,721537506022013,0
595,721537506022014,0


In [92]:
final_df[final_df['GEOID20'].map(len)==14]

,GEOID20,address_count


In [93]:
final_df['GEOID20'].map(len).value_counts()

15    8174038
Name: GEOID20, dtype: int64

In [96]:
final_df.to_csv('../data/coverage_report.csv', index=False)